In [1]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F
import torch.optim as optim

# 1.定义神经网络

In [2]:
class NetA(nn.Module):
    
    def __init__(self):
        super(NetA, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
class NetB(nn.Module):
    
    def __init__(self):
        super(NetB, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

# 2.初始化神经网络实例

In [3]:
# model
net = NetA()
print(f"Model:\n==========\n {net}")

Model:
 NetA(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# 3.初始化优化器

In [4]:
# initialize optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)
print(f"Optimizer:\n==========\n {optimizer}")

Optimizer:
 SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


# 4.state_dict

## 4.1 net.state_dict()

In [5]:
# print model's state_dict
print("Model's state_dict:\n======================")
for param_tensor in net.state_dict():
    print(f"{param_tensor}, \t, {net.state_dict()[param_tensor].size()}")

Model's state_dict:
conv1.weight, 	, torch.Size([6, 3, 5, 5])
conv1.bias, 	, torch.Size([6])
conv2.weight, 	, torch.Size([16, 6, 5, 5])
conv2.bias, 	, torch.Size([16])
fc1.weight, 	, torch.Size([120, 400])
fc1.bias, 	, torch.Size([120])
fc2.weight, 	, torch.Size([84, 120])
fc2.bias, 	, torch.Size([84])
fc3.weight, 	, torch.Size([10, 84])
fc3.bias, 	, torch.Size([10])


### 4.2 optimizer.state_dict()

In [6]:
# print optimizer's state_dict
print("Optimizer's state_dict:\n=======================")
for var_name in optimizer.state_dict():
    print(f"{var_name} \t, {optimizer.state_dict()[var_name]}")

Optimizer's state_dict:
state 	, {}
param_groups 	, [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


# 5.保存、加载模型

## 5.1 保存、加载用于推理的模型

### 5.1.1 保存、加载模型 state_dict

In [7]:
PATH = "./models/state_dict_model.pt"

In [8]:
# model save
torch.save(net.state_dict(), PATH)

In [9]:
# model load
model = NetA()
model.load_state_dict(torch.load(PATH)) # 必须在将保存的 state_dict 传递给 load_state_dict 之前反序列化它
model.eval()

NetA(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### 5.1.2 保存、加载整个模型

In [10]:
PATH = "./models/entire_model.pt"

In [11]:
# model save
torch.save(net, PATH)

In [12]:
# model load
model = torch.load(PATH)
model.eval()

NetA(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 5.2 保存和加载常规 checkpoint

In [13]:
PATH = "./models/checkpoint_model.pt"
EPOCH = 5
LOSS = 0.4

# model save
torch.save({
    'epoch': EPOCH,
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': LOSS,
    }, PATH)

In [14]:
# model load
model = NetA()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
epoch = checkpoint["epoch"]
loss = checkpoint["loss"]

model.eval()
# - or - 
model.train()

NetA(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 5.3 保存和加载多个模型

In [15]:
# models
netA = NetA()
netB = NetA()

# optimizers
optimizerA = optim.SGD(netA.parameters(), lr = 0.001, momentum = 0.9)
optimizerB = optim.SGD(netB.parameters(), lr = 0.001, momentum = 0.9)

In [16]:
# path of model save
PATH = "./models/model.pt"

# model save
torch.save({
    "modelA_state_dict": netA.state_dict(),
    "modelB_state_dict": netB.state_dict(),
    "optimizerA_state_dict": optimizerA.state_dict(),
    "optimizerB_state_dict": optimizerB.state_dict(),
}, PATH)

In [17]:
# model load
modelA = NetA()
modelB = NetA()
optimModelA = optim.SGD(modelA.parameters(), lr = 0.001, momentum = 0.9)
optimModelB = optim.SGD(modelB.parameters(), lr = 0.001, momentum = 0.9)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint["modelA_state_dict"])
modelB.load_state_dict(checkpoint["modelB_state_dict"])
optimizerA.load_state_dict(checkpoint["optimizerA_state_dict"])
optimizerB.load_state_dict(checkpoint["optimizerB_state_dict"])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()

NetA(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 5.4 使用不同模型参数的热启动模型

In [18]:
PATH = "./models/A_model.pt"

# save model A
torch.save(netA.state_dict(), PATH)

# load model B
netB.load_state_dict(torch.load(PATH), strict = False)

<All keys matched successfully>

## 5.5 跨设备保存、加载模型

### 5.5.1 在 GPU 上保存、在 CPU 上加载

In [19]:
PATH = "./models/model.pt"s

# model save
torch.save(net.state_dict(), PATH)

# model load
device = torch.device("cpu")
model = NetA()
model.load_state_dict(torch.load(PATH, map_location = device))

<All keys matched successfully>

### 5.5.2 在 GPU 上保存、在 GPU 上加载

In [21]:
PATH = "./models/model.pt"

# model save
torch.save(net.state_dict(), PATH)

# model load
device = torch.device("cuda")
model = NetA()
model.load_state_dict(torch.load(PATH))
model.to(device)

### 5.5.3 在 CPU 上保存、在 GPU 上加载

In [ ]:
PATH = "./models/model.pt"

# model save
torch.save(net.state_dict(), PATH)

# model load
device = torch.device("cuda")
model = NetA()
model.load_state_dict(torch.load(PATH, map_location = "cuda:0"))
model.to(device)

### 5.5.4 保存 torch.nn.DataParallel 模型

In [ ]:
PATH = "./models/model.pt"

# model save
torch.save(net.module.state_dict(), PATH)

# load to any divce
torch.load(PATH)

# - or -
model = NetA()
model.load_state_dict(torch.load())